# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

def populate_table(tablename, feature_names):
    iris = datasets.load_iris()
    X = iris.data  
    N = X.shape[0]
    y = iris.target.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm'],
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/iris.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/iris.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,TGT
114,114,114,5.8,2.8,5.1,2.4,2
74,74,74,6.4,2.9,4.3,1.3,1
9,9,9,4.9,3.1,1.5,0.1,0
88,88,88,5.6,3.0,4.1,1.3,1
25,25,25,5.0,3.0,1.6,0.2,0
5,5,5,5.4,3.9,1.7,0.4,0
48,48,48,5.3,3.7,1.5,0.2,0
117,117,117,7.7,3.8,6.7,2.2,2
83,83,83,6.0,2.7,5.1,1.6,1
105,105,105,7.6,3.0,6.6,2.1,2


## Training a Model

In [5]:


# train any scikit model on the iris dataset
    
from sklearn.calibration import CalibratedClassifierCV
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier(random_state=1960)
clf = CalibratedClassifierCV(dec_tree, method='sigmoid', cv=5)

clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


/home/antoine/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


CalibratedClassifierCV(base_estimator=DecisionTreeClassifier(random_state=1960),
                       cv=5)

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	std::vector<std::any> get_classes(){
		std::vector<std::any> lClasses = { 0, 1, 2 };

		return lClasses;
	}

	namespace BestEstimator_0 {
	
		std::vector<std::any> get_classes(){
			std::vector<std::any> lClasses = { 0, 1, 2 };
	
			return lClasses;
		}
	
		typedef std::vector<double> tNodeData;
		std::map<int, tNodeData> Decision_Tree_Node_data = {
				{ 1 ,  {1.0, 0.0, 0.0 }} ,
				{ 3 ,  {0.0, 1.0, 0.0 }} ,
				{ 6 ,  {0.0, 1.0, 0.0 }} ,
				{ 8 ,  {0.0, 0.0, 1.0 }} ,
				{ 10 ,  {0.0, 1.0, 0.0 }} ,
				{ 11 ,  {0.0, 0.0, 1.0 }} ,
				{ 14 ,  {0.0, 1.0, 0.0 }} ,
				{ 15 ,  {0.0, 0.0, 1.0 }} ,
				{ 16 ,  {0.0, 0.0, 1.0 }} 
		};
		
	
		int get_decision_tree_node_index(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3) {
			int lNodeIndex = (Feature_3 <= 0.800000011920929) ? ( 1 ) : ( (Feature_2 <= 4.75) ? ( 3 ) : ( (Feature_3 <= 1.75) ? ( (Feature_2 <= 4.950000047683716) ? ( 6 ) : ( (Feature_3 <= 1.550000011920929) ? ( 8 ) : ( (Feature_0 <

In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:1000])
        print(result2[-1000:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp__" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/iris2.csv" , ["Feature_0", "Feature_1", "Feature_2", "Feature_3"])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/iris2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp__139972209238896.exe /tmp/sklearn2sql_cpp__139972209238896.cpp'
idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
0,,,,0.86756071609704,0.06553146640495,0.06690781749801,-0.14206977987306,-2.72522484856857,-2.70443946519187,0,0.86756071609704
1,,,,0.86756071609704,0.06553146640495,0.06690781749801,-0.14206977987306,-2.72522484856857,-2.70443946519187,0,0.86756071609704
2,,,,0.86756071609704,0.06553146640495,0.06690781749801,-0.14206977987306,-2.72522484856857,-2.70443946519187,0,0.86756071609704
3,,,,0.86756071609704,0.06553146640495,0.06690781749801,-0.14206977987306,-2.72522484856857,-2.70443946519187,0,0.86756071609704
4,,,,0.86756071609704,0.06553146640495,0.06690781749801,-0.14206977987306,-2.72522484856857,-2.70443946519187,0,0.86756071609704
5,,,,0.86756071609704,0.06553146640495,0.06690781749801,-0.14206977987306,-2.72522484856857,-2.70443946

In [10]:
cpp_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
114,114,NaN,NaN,NaN,0.046194,0.072551,0.881255,-3.074911,-2.623461,-0.126408,2,0.881255
74,74,NaN,NaN,NaN,0.046295,0.880713,0.072992,-3.072721,-0.127023,-2.617411,1,0.880713
9,9,NaN,NaN,NaN,0.867561,0.065531,0.066908,-0.142070,-2.725225,-2.704439,0,0.867561
88,88,NaN,NaN,NaN,0.046295,0.880713,0.072992,-3.072721,-0.127023,-2.617411,1,0.880713
25,25,NaN,NaN,NaN,0.867561,0.065531,0.066908,-0.142070,-2.725225,-2.704439,0,0.867561
5,5,NaN,NaN,NaN,0.867561,0.065531,0.066908,-0.142070,-2.725225,-2.704439,0,0.867561
48,48,NaN,NaN,NaN,0.867561,0.065531,0.066908,-0.142070,-2.725225,-2.704439,0,0.867561
117,117,NaN,NaN,NaN,0.046194,0.072551,0.881255,-3.074911,-2.623461,-0.126408,2,0.881255
83,83,NaN,NaN,NaN,0.046646,0.716687,0.236667,-3.065171,-0.333116,-1.441101,1,0.716687
105,105,NaN,NaN,NaN,0.046194,0.072551,0.881255,-3.074911,-2.623461,-0.126408,2,0.881255


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_log_proba = pd.DataFrame(np.log(clf.predict_proba(X)), columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,NaN,NaN,NaN,0.046194,0.072551,0.881255,-3.074911,-2.623461,-0.126408,2
74,74,NaN,NaN,NaN,0.046295,0.880713,0.072992,-3.072721,-0.127023,-2.617411,1
9,9,NaN,NaN,NaN,0.867561,0.065531,0.066908,-0.142070,-2.725225,-2.704439,0
88,88,NaN,NaN,NaN,0.046295,0.880713,0.072992,-3.072721,-0.127023,-2.617411,1
25,25,NaN,NaN,NaN,0.867561,0.065531,0.066908,-0.142070,-2.725225,-2.704439,0
5,5,NaN,NaN,NaN,0.867561,0.065531,0.066908,-0.142070,-2.725225,-2.704439,0
48,48,NaN,NaN,NaN,0.867561,0.065531,0.066908,-0.142070,-2.725225,-2.704439,0
117,117,NaN,NaN,NaN,0.046194,0.072551,0.881255,-3.074911,-2.623461,-0.126408,2
83,83,NaN,NaN,NaN,0.046646,0.716687,0.236667,-3.065171,-0.333116,-1.441101,1
105,105,NaN,NaN,NaN,0.046194,0.072551,0.881255,-3.074911,-2.623461,-0.126408,2


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
114,114,NaN,NaN,NaN,0.046194,0.072551,0.881255,-3.074911,-2.623461,-0.126408,...,NaN,NaN,0.046194,0.072551,0.881255,-3.074911,-2.623461,-0.126408,2,0.881255
74,74,NaN,NaN,NaN,0.046295,0.880713,0.072992,-3.072721,-0.127023,-2.617411,...,NaN,NaN,0.046295,0.880713,0.072992,-3.072721,-0.127023,-2.617411,1,0.880713
9,9,NaN,NaN,NaN,0.867561,0.065531,0.066908,-0.142070,-2.725225,-2.704439,...,NaN,NaN,0.867561,0.065531,0.066908,-0.142070,-2.725225,-2.704439,0,0.867561
88,88,NaN,NaN,NaN,0.046295,0.880713,0.072992,-3.072721,-0.127023,-2.617411,...,NaN,NaN,0.046295,0.880713,0.072992,-3.072721,-0.127023,-2.617411,1,0.880713
25,25,NaN,NaN,NaN,0.867561,0.065531,0.066908,-0.142070,-2.725225,-2.704439,...,NaN,NaN,0.867561,0.065531,0.066908,-0.142070,-2.725225,-2.704439,0,0.867561
5,5,NaN,NaN,NaN,0.867561,0.065531,0.066908,-0.142070,-2.725225,-2.704439,...,NaN,NaN,0.867561,0.065531,0.066908,-0.142070,-2.725225,-2.704439,0,0.867561
48,48,NaN,NaN,NaN,0.867561,0.065531,0.066908,-0.142070,-2.725225,-2.704439,...,NaN,NaN,0.867561,0.065531,0.066908,-0.142070,-2.725225,-2.704439,0,0.867561
117,117,NaN,NaN,NaN,0.046194,0.072551,0.881255,-3.074911,-2.623461,-0.126408,...,NaN,NaN,0.046194,0.072551,0.881255,-3.074911,-2.623461,-0.126408,2,0.881255
83,83,NaN,NaN,NaN,0.046646,0.716687,0.236667,-3.065171,-0.333116,-1.441101,...,NaN,NaN,0.046646,0.716687,0.236667,-3.065171,-0.333116,-1.441101,1,0.716687
105,105,NaN,NaN,NaN,0.046194,0.072551,0.881255,-3.074911,-2.623461,-0.126408,...,NaN,NaN,0.046194,0.072551,0.881255,-3.074911,-2.623461,-0.126408,2,0.881255


In [14]:
condition = (cpp_skl_join.Decision_cpp != cpp_skl_join.Decision_skl)
cpp_skl_join[condition]


,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
